In [ ]:
# ensure required packages are installed in this notebook environment
%pip install --quiet pandas requests

import pandas as pd      # primary library for managing and manipulating data
import requests          # library for calling API endpoints